In [1]:
import requests
from bs4 import BeautifulSoup

def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk:
                f.write(chunk)
    return local_filename


httplink = [
"https://www.dmhy.org/topics/view/693073_04_Aharen-san_wa_Hakarenai_S2_01_1080p.html",
"https://www.dmhy.org/topics/view/693539_04_Aharen-san_wa_Hakarenai_S2_02_1080p.html",
]


torrentid = []
magnetlink = []


for httpmount in httplink:
  response = requests.get(
      httpmount)
  soup = BeautifulSoup(response.text, "html.parser")

  titles = soup.find_all("div", id="tabs-1")
  for title in titles:
    myto = "https:"+title.find_all("a")[0].get('href')
    torrentid.append(myto[31:])

    download_file(myto)

    target = ""
    head = 100
    mag1 = title.find_all("p")[0].find_all("p")[0].text[10:]
    mag2 = title.find_all("p")[0].find_all("p")[1].text[16:]
    magnetlink.append([mag1,mag2])

for i in torrentid:
  print(i)
for i in magnetlink:
  print(i)


69e2bf938eb5b88726e252b7a93d2fc15544045b.torrent
985b937fed1f92ed35987f635cf2864c5d610824.torrent
['magnet:?xt=urn:btih:NHRL7E4OWW4IOJXCKK32SPJPYFKUIBC3', 'magnet:?xt=urn:btih:69e2bf938eb5b88726e252b7a93d2fc15544045b']
['magnet:?xt=urn:btih:TBNZG77ND6JO2NMYP5RVZ4UGJROWCCBE', 'magnet:?xt=urn:btih:985b937fed1f92ed35987f635cf2864c5d610824']


In [2]:
import time
from IPython.display import display
import ipywidgets as widgets
import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []


for iii in range(len(torrentid)):
  ses = lt.session()
  ses.listen_on(6881, 6891)
  downloads = []
  print(lt.torrent_info(torrentid[iii]))
  params = {
      "save_path": "/workspaces/tools/Torrent",
      "ti": lt.torrent_info(torrentid[iii]),
  }
  downloads.append(ses.add_torrent(params))

  params = {"save_path": "/workspaces/tools/Torrent"}
  downloads.append(lt.add_magnet_uri(ses, magnetlink[iii][0], params))
  downloads.append(lt.add_magnet_uri(ses, magnetlink[iii][1], params))

  state_str = [
      "queued",
      "checking",
      "downloading metadata",
      "downloading",
      "finished",
      "seeding",
      "allocating",
      "checking fastresume",
  ]

  layout = widgets.Layout(width="auto")
  style = {"description_width": "initial"}
  download_bars = [
      widgets.FloatSlider(
          step=0.01, disabled=True, layout=layout, style=style
      )
      for _ in downloads
  ]
  display(*download_bars)

  try:
    while downloads:
        next_shift = 0
        for index, download in enumerate(downloads[:]):
            bar = download_bars[index + next_shift]
            if not download.is_seed():
                s = download.status()

                bar.description = " ".join(
                    [
                        download.name(),
                        str(s.download_rate / 1000),
                        "kB/s",
                        state_str[s.state],
                    ]
                )
                bar.value = s.progress * 100
            else:
                next_shift -= 1
                ses.remove_torrent(download)
                downloads.remove(download)
                bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
                download_bars.remove(bar)
                print(download.name(), "complete")
        time.sleep(1)
  except:
    continue

/tmp/ipykernel_11097/178444606.py:7: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)
/tmp/ipykernel_11097/178444606.py:13: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)
/tmp/ipykernel_11097/178444606.py:23: DeprecationWarning: add_magnet_uri() is deprecated
  downloads.append(lt.add_magnet_uri(ses, magnetlink[iii][0], params))
/tmp/ipykernel_11097/178444606.py:24: DeprecationWarning: add_magnet_uri() is deprecated
  downloads.append(lt.add_magnet_uri(ses, magnetlink[iii][1], params))


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

/tmp/ipykernel_11097/178444606.py:52: DeprecationWarning: is_seed() is deprecated
  if not download.is_seed():
/tmp/ipykernel_11097/178444606.py:57: DeprecationWarning: name() is deprecated
  download.name(),


...

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

[Nekomoe kissaten][Aharen-san wa Hakarenai S2][02][1080p][CHT].mp4 complete


Bad pipe message: %s [b'0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nHost: localhost:45631\r\nUs', b'-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.']
Bad pipe message: %s [b'0.0 Safari/537.36\r\nAccept-Encoding: gzip, defla']
Bad pipe message: %s [b', br, zstd\r\nAccept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7\r\nReferer: https://curly-space-happiness-5p4xqj66wv', b'px5q.github.dev/\r\nX-Request-ID: b81daae02153d2f021b3f17f92cd9953\r\nX-Real-IP: 223.137.10.22\r\nX-Forwarde']
Bad pipe message: %s [b'Port: 443\r\nX-Forwarded-Scheme: https\r\nX-Original-URI: /\r\nX-Scheme: https\r\nsec-ch-ua: "Not)A;Brand";', b'"8", "Chromium";v="138", "Google Chrome";v="138"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-ua-platform: "Windows"\r\nsec-fetch-site', b'same-site\r\nsec-fetch-mode: navigate\r\nsec-fetch-user: ?1\r\n']
